In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "yoked_from_scratch_plots/"
create_directory(data_directory)
create_directory("models")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS, YOKED_TRAINING_COLS
from isaac.training import training_loop, evaluate_saved_model
from isaac.models import MultiBranchModel, initialise_model

In [4]:
BATCH_SIZE = 128
EPOCHS = 2
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 1800

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
exp1_trials = read_dataset("data/yoked_trials_exp1.h5")
exp2_mass_trials = read_dataset("data/mass_trials_exp2.h5")
exp2_force_trials = read_dataset("data/force_trials_exp2.h5")

exp1_trials = [trial[:1801] for trial in exp1_trials]
all_trials = np.array(exp1_trials + exp2_mass_trials + exp2_force_trials)

train_indices, val_indices = train_test_split(np.arange(len(all_trials)), test_size=0.1)
val_indices, test_indices = train_test_split(val_indices, test_size=0.5)

train_trials = all_trials[train_indices]
val_trials = all_trials[val_indices]
test_trials = all_trials[test_indices]

len(train_trials), len(val_trials), len(test_indices)

100%|██████████| 399/399 [00:06<00:00, 57.37it/s]


(915, 51, 51)

## Define model, loss and optimizer

## Train model and plot loss and accuracy

In [6]:
def get_best_model_and_its_accuracy(accuracies, model, best_model, best_accuracy):
    model_accuracy = max(accuracies)
    print(model_accuracy)
    if model_accuracy > best_accuracy:
        return deepcopy(model), model_accuracy

    return best_model, best_accuracy

In [7]:
stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True, training_columns=BASIC_TRAINING_COLS,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA, 
                                  device=device)

best_mass_model_overall = None
best_mass_accuracy = 0.
best_force_model_overall = None
best_force_accuracy = 0.

for seed in [0, 42, 72]:
    df = pd.DataFrame()


    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1])
    
    best_mass_model_overall, best_mass_accuracy = get_best_model_and_its_accuracy(val_accuracies[:, 0], 
                                                                                  best_mass_model, 
                                                                                  best_mass_model_overall,
                                                                                  best_mass_accuracy)

    best_force_model_overall, best_force_accuracy = get_best_model_and_its_accuracy(val_accuracies[:, 1], 
                                                                                    best_force_model,
                                                                                    best_force_model_overall,
                                                                                    best_force_accuracy)

    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
# stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 51/51 [00:00<00:00, 629.58it/s]
Train_loss: ([1.09587441 1.09019125]) Train_acc: ([40.43715847 40.87431694]) Val_acc: ([37.25490196 43.1372549 ]): 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]
  0%|          | 0/2 [00:00<?, ?it/s]

37.254901960784316
43.13725490196079


Train_loss: ([1.10016601 1.09213865]) Train_acc: ([39.78142077 38.90710383]) Val_acc: ([43.1372549  45.09803922]): 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]
  0%|          | 0/2 [00:00<?, ?it/s]

43.13725490196079
45.09803921568628


Train_loss: ([1.09578782 1.09795712]) Train_acc: ([39.78142077 36.39344262]) Val_acc: ([35.29411765 45.09803922]): 100%|██████████| 2/2 [00:04<00:00,  2.33s/it]

35.294117647058826
45.09803921568628


In [8]:
best_mass_accuracy, best_force_accuracy

(43.13725490196079, 45.09803921568628)

## Save model and scaler

torch.save(best_mass_model_overall.state_dict(), "models/yoked_mass_dual_model.pt")
torch.save(best_force_model_overall.state_dict(), "models/yoked_force_dual_model.pt")

joblib.dump(scaler, "scalers/yoked_dual_scaler.sk")

# Evaluating models on test data and CMs

def plot(predicted, i, class_columns, test_loader):
    predicted = [pred.cpu() for pred in predicted]
    Y_test = np.concatenate([y[:, i].cpu().numpy() for x, y in test_loader])

    # plot_confusion_matrix(Y_test, predicted, classes=class_columns, normalize=False)
    plot_confusion_matrix(Y_test, predicted, classes=class_columns, normalize=True, )

test_loader, _ = prepare_dataset([test_trials], class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                 training_columns=BASIC_TRAINING_COLS, multiclass=True, scaler=scaler,
                                 normalise_data=True)
mass_accuracy, _, mass_predicted, _ = evaluate(best_mass_model_overall, test_loader, return_predicted=True, seq_end=1800, step_size=3)
print(mass_accuracy)
plot(mass_predicted, 0, MASS_CLASS_COLS, test_loader)
plt.savefig("yoked_from_scratch_plots/mass_cm.pdf")

test_loader, _ = prepare_dataset([test_trials], class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                 training_columns=BASIC_TRAINING_COLS, multiclass=True, scaler=scaler,
                                 normalise_data=True)
_, force_accuracy, _, force_predicted = evaluate(best_force_model_overall, test_loader, return_predicted=True, seq_end=1800, step_size=3)
print(force_accuracy)
plot(force_predicted, 1, FORCE_CLASS_COLS, test_loader)
plt.savefig("yoked_from_scratch_plots/force_cm.pdf")